In [2]:
import os
import pandas as pd
import numpy as np
import json
import hashlib

# Project paths
PROJECT_ROOT = r'C:\Users\shrey\Desktop\Projects\Explainable Price Anomaly Detector for Indian Second-hand Marketplace'
DATA_PATH = os.path.join(PROJECT_ROOT, 'data', 'cleaned_with_age.csv')
OUTPUT_PATH = os.path.join(PROJECT_ROOT, 'data', 'cleaned_engineered.csv')
METADATA_PATH = os.path.join(PROJECT_ROOT, 'reports', 'cleaning_metadata.json')
os.makedirs(os.path.join(PROJECT_ROOT, 'reports'), exist_ok=True)

# Load dataset
df = pd.read_csv(DATA_PATH, low_memory=False)
df.columns = df.columns.str.strip().str.lower()
print('Initial shape:', df.shape)

# 1. Drop columns with >80% missing values
missing_percent = df.isnull().mean() * 100
drop_cols = missing_percent[missing_percent > 80].index.tolist()
print(f'Columns with >80% missing values (to be dropped): {drop_cols}')
df = df.drop(columns=drop_cols)
print(f'Shape after dropping columns: {df.shape}')

# 2. Impute missing numerical values with medians
num_cols = ['km', 'car_age', 'max power delivered', 'alloy wheel size', 'wheel base', 
            'no of cylinder', 'length', 'width', 'height', 'top speed', 'acceleration', 
            'kerb weight', 'gross weight', 'front tread', 'rear tread', 'turning radius', 
            'cargo volume', 'max torque delivered', 'max power at', 'max torque at', 'bore']

available_num_cols = [c for c in num_cols if c in df.columns]
medians = df[available_num_cols].median()
print('\nMedian values for imputation:\n', medians)

for col in available_num_cols:
    df[col] = df[col].fillna(medians[col])
    print(f'Imputed {col} with median {medians[col]}')

# 3. Standardize categorical columns
cat_cols = ['fuel', 'transmission', 'city', 'oem', 'model', 'variant', 'color', 'engine type', 
            'owner_type', 'drive type', 'steering type', 'front brake type', 'rear brake type', 'tyre type']

for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown').str.lower().str.strip()
        print(f'Standardized {col} (filled NaN with "Unknown")')

# 4. Handle outliers (clip km and listed_price)
df['km'] = df['km'].clip(lower=df['km'].quantile(0.01), upper=df['km'].quantile(0.99))
df['listed_price'] = df['listed_price'].clip(lower=df['listed_price'].quantile(0.01), 
                                             upper=df['listed_price'].quantile(0.99))
print(f'\nClipped km to [{df["km"].min():,.0f}, {df["km"].max():,.0f}]')
print(f'Clipped listed_price to [{df["listed_price"].min():,.0f}, {df["listed_price"].max():,.0f}]')

# 5. Add feature: km_per_year
df['km_per_year'] = df['km'] / (df['car_age'] + 1e-5)  # Avoid division by zero
print('Added feature: km_per_year')

# 6. Remove invalid rows (e.g., non-positive prices)
initial_rows = df.shape[0]
df = df[df['listed_price'] > 0]
print(f'Removed {initial_rows - df.shape[0]} rows with non-positive prices')
print(f'Final shape: {df.shape}')

# Verify missing values
print('\nMissing values after cleaning:\n', df.isnull().sum().sort_values(ascending=False).head(10))

# Save cleaned and engineered dataset
df.to_csv(OUTPUT_PATH, index=False)
print(f'Cleaned and engineered dataset saved to {OUTPUT_PATH}')

# Generate dataset checksum
def file_md5(file_path):
    with open(file_path, 'rb') as f:
        return hashlib.md5(f.read()).hexdigest()

dataset_checksum = file_md5(OUTPUT_PATH)

# Save metadata
metadata = {
    "initial_shape": str(initial_rows),
    "final_shape": str(df.shape),
    "dropped_columns": drop_cols,
    "imputed_columns": available_num_cols,
    "categorical_standardized": [c for c in cat_cols if c in df.columns],
    "new_features": ["km_per_year"],
    "checksum": dataset_checksum
}

with open(METADATA_PATH, 'w') as f:
    json.dump(metadata, f, indent=4)

print(f'Metadata saved to {METADATA_PATH}')

# Update README
readme_content = f"""
# Cleaning and Feature Engineering Summary
- Started with dataset: {initial_rows} rows.
- Dropped columns with >80% missing values: {drop_cols}.
- Imputed numerical columns with medians: {available_num_cols}.
- Standardized categorical columns, filled NaN with 'Unknown'.
- Clipped outliers in km and listed_price (1st/99th percentiles).
- Added feature: km_per_year = km / car_age.
- Removed rows with non-positive prices.
- Final dataset shape: {df.shape}, checksum: {dataset_checksum}.
- Next steps: Train baseline model for price prediction.
"""
with open(os.path.join(PROJECT_ROOT, 'README.md'), 'a') as f:
    f.write(readme_content)
print('README.md updated with cleaning summary.')

Initial shape: (37813, 67)
Columns with >80% missing values (to be dropped): ['ground clearance unladen', 'stroke']
Shape after dropping columns: (37813, 65)

Median values for imputation:
 km                      56726.00
car_age                     9.00
max power delivered        88.50
alloy wheel size           16.00
wheel base               2520.00
no of cylinder              4.00
length                   3995.00
width                    1710.00
height                   1530.00
top speed                 168.56
acceleration               13.20
kerb weight              1066.00
gross weight             1595.00
front tread              1485.00
rear tread               1493.00
turning radius              5.10
cargo volume              350.00
max torque delivered      145.00
max power at             5500.00
max torque at            3375.00
bore                       73.00
dtype: float64
Imputed km with median 56726.0
Imputed car_age with median 9.0
Imputed max power delivered with median